In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

plt.style.use('fivethirtyeight')
%config InlineBackend.figure_formats = ['svg']

In [7]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])
zp_mean = zp.mean()
ks_mean = ks.mean()

cov = ((zp - zp_mean) * (ks - ks_mean)).sum() / (zp.shape[0] - 1)
numpy_cov = np.cov(zp, ks)

cov, numpy_cov

(10175.37777777778,
 array([[ 3882.93333333, 10175.37777778],
        [10175.37777778, 33854.32222222]]))